Deep contextualized word representation has drawn wide attention because of state-of-the-art performances in downstream tasks. Contextualized embeddings can capture not only word-level information but also multi-sense information, thus improving the results in sentiment analysis, SQuad and etc. However, the language adopted in the [Elmo](https://allennlp.org/elmo) model were biLSTMs which contained a huge number of parameters, it was less likely for small labs to train and run such experiments.


In this project, we intend to make use of CNN language model in learning efficient word representations for sentiment analysis. We train a language model based on [Gated CNN architecture](https://arxiv.org/abs/1612.08083) proposed by Yann Daulphin, then do sentiment analysis with embeddings generated by the language model.

The language model training dataset is 1-billion-word-language.

In [1]:
import tensorflow as tf
import os
import time

from model import *
from data_utils import data_helper
from conf_utils import *

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Initialize the configuration and prepare data batches

In [2]:
class config:
    vocab_size = 2000
    embedding_size = 200
    filter_size = 64
    num_layers = 1
    block_size = 2
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 32
    epochs = 5
    num_sampled = 64
    learning_rate = 0.0001
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    #data_dir = "data/texts/reviews/movie_reviews"
    data_dir = "data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled"

In [3]:
#Initialize configuration files
conf = prepare_conf(config)

In [10]:
#Create data batches for language model training
dh = data_helper(conf)
x_batches, y_batches, word_to_idx, idx_to_word = dh.prepare_data()

In [5]:
conf.text_size

20

In [6]:
len(x_batches[0][0])

20

## Train a CNN-based language model

In [4]:
conf.text_size

20

In [14]:
#Create a language model
#Note we need to save the models for subsequent tasks
model = GatedCNN(conf)
saver = tf.train.Saver(tf.trainable_variables())
print("Started Model Training...")

Tensor("Reshape:0", shape=(640, 200), dtype=float32)
Started Model Training...


In [9]:
#from RNN import RNN
#model = RNN(conf)
#saver = tf.train.Saver(tf.trainable_variables())

In [10]:
#labels.shape

In [7]:
conf.num_batches

32895

In [8]:
batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(conf.summary_path, graph=sess.graph)

    #if os.path.exists(conf.ckpt_file):
        #saver.restore(sess, conf.ckpt_file)
        #print("Model Restored")

    for i in np.arange(conf.epochs):
        start = time.time()
        for j in np.arange(conf.num_batches):
        #for j in np.arange(21):
            inputs, labels, batch_idx = dh.get_batch(x_batches, y_batches, batch_idx)
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.y:labels})
            if j%100 == 0:
                print(l)
        end = time.time()
        print("Epoch: %.2f, Time: %.2f,  Loss: %.2f"%(i, end-start, l))

        if i % 2 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.y:labels})
            print("Perplexity: %.2f"%perp)
            saver.save(sess, conf.ckpt_file)

        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.y:labels})
        summary_writer.add_summary(summaries, i)

228.28345
208.6025
158.5471
100.787155
104.19407
112.967186
63.305267
84.85893
60.24416
39.28103
40.074116
25.748016
34.781715
29.925396
34.351105
29.201511
15.204874
17.922197
14.43259
16.130964
9.519644
4.854629
15.488127
17.754185
5.130473
11.15888
4.7705493
4.6013713
4.7350683
4.91406
6.793849
4.506275
4.7903934
5.2425947
6.6091423
4.25373
4.608274
4.285217
5.8390694
4.3712454
4.191039
4.2742143
4.2655087
4.107478
4.2508354
4.176041
4.422719
4.129487
4.24267
4.236827
4.114705
4.1388817
4.2875743
4.259644
4.15853
4.2453675
4.203578
4.0906987
4.2734327
4.205821
4.165262
4.248075
4.251493
4.251256
4.081624
4.126757
4.1385245
4.069039
4.212142
4.1817803
4.1815014
4.183185
4.236308
4.125826
4.1625557
4.1035523
4.16032
4.057517
4.1799
4.1593404
4.158732
4.1533785
4.0161977
4.0019574
4.221444
4.0739126
4.2144003
4.1436186
4.108012
4.2313533
4.1696095
4.009533
4.113341
4.1602373
4.1060157
4.182256
4.0892563
3.9855568
4.059971
4.1261187
4.133526
4.176199
4.12121
4.0554338
4.1574736
4.031247

3.8146527
3.8352158
3.8189282
3.8980064
3.7937012
3.8909516
3.8552842
3.8255928
3.8695092
3.865809
3.8576405
3.8010106
3.8385587
3.7396502
3.7994144
3.7645729
4.048583
4.000402
3.686047
3.9688983
3.9008183
3.9297683
3.7904804
3.9080608
3.859784
3.760014
3.7979932
3.9005756
3.742144
3.9099407
3.7749634
3.8058465
3.7907271
3.9299178
3.8839583
3.8878617
3.8886814
3.7044272
3.7893963
3.9048645
3.793591
3.9308686
3.9838052
3.735489
3.859272
3.9235027
3.8708458
3.897118
3.7819297
3.7730942
3.9242566
3.914508
3.791121
3.7637558
3.8108432
3.9001842
3.9082017
3.8188434
3.8070285
3.788227
3.9661782
3.9224105
3.7986386
3.8654666
3.9685674
3.7913852
3.781276
3.8970933
3.8334584
3.7797897
3.8435364
3.8779728
3.9667327
3.9638188
3.8470745
3.8016975
3.85028
3.8261209
3.9016156
3.7042305
3.8667111
3.8518646
3.9682987
3.7704518
3.8392262
3.9365964
3.8708825
3.8713691
3.8546124
3.7848148
3.8357296
3.7532356
3.8482394
3.847215
3.69712
3.7927868
3.9138057
3.951427
3.8880935
3.9020653
3.9058297
3.8912673
3

## Train a RNN model

## Sentiment Analysis

In this part, we need to use other datasets for sentiment analysis, like the one of SemEval2013.

In [1]:
#Use spacy tokenize sentences into words
#!pip install spacy
#!python -m spacy download en
#import spacy
#nlp = spacy.load('en')
#def sent_split(sent):
    #words = []
    #sent = nlp(sent.strip())
    #for w in sent:
        #words.append(w.text.lower())
    #return words

In [4]:
file_train = 'data/semeval/downloaded.tsv'
file_dev = 'data/semeval/dev_downloaded.tsv'
file_test = 'data/semeval/test.txt'
with open(file_train) as f:
    tweets_train = f.readlines()
with open(file_dev) as f:
    tweets_dev = f.readlines()
with open(file_test) as f:
    tweets_test = f.readlines()
    


#Filter empty tweets
def is_available(text):
    if 'Not Available' in text:
        return False
    if '\t"objective' in text:
        return False
    if '\t"neutral' in text:
        return False
    if '\tobjective' in text:
        return False
    if '\tneutral' in text:
        return False
    return True


In [5]:
tweets_train = list(filter(is_available, tweets_train))
tweets_dev = list(filter(is_available, tweets_dev))
tweets_test = list(filter(is_available, tweets_test))

In [6]:
tweets_train = [item.split('\t') for item in tweets_train]
tweets_dev = [item.split('\t') for item in tweets_dev]
tweets_test = [item.split('\t') for item in tweets_test]
_, _, y_train, text_train = list(zip(*tweets_train))
_, _, y_dev, text_dev = list(zip(*tweets_dev))
_, _, y_test, text_test = list(zip(*tweets_test))


In [7]:
text_train, y_train = list(text_train), list(y_train)
text_dev, y_dev = list(text_dev), list(y_dev)
text_test, y_test = list(text_test), list(y_test)
y_test = ['"' + item + '"' for item in y_test]

In [8]:
#Encode the labels to numbers
from sklearn import preprocessing
label_encode = preprocessing.LabelEncoder()  # 建立模型
y_train = label_encode.fit_transform(y_train)
y_dev = label_encode.transform(y_dev)
y_test = label_encode.transform(y_test)

## Twitter Data Preprocessing

In [89]:
import numpy as np
import spacy
nlp = spacy.load('en')
class generate_samples:
    '''
    Generate samples of training data or testing data for data analysis
    '''
    def __init__(self, data, labels, word_to_idx, max_sent_len=20, is_training=True):
        '''
        Args:
        data: numpy
        labels: numpy
        '''
        self.data = data
        self.labels = labels
        self.word_to_idx = word_to_idx
        self.is_training = is_training
        self.max_sent_len = max_sent_len
        self.index = 0
        
    def sent_split(self, sent):
        '''
        Split a sentence into tokens
        '''
        words = []
        sent = nlp(sent.strip())
        for w in sent:
            words.append(w.text.lower())
        return words
        
    def generate_samples(self, sents, labels, batch_size=64):
        '''
        Select a batch_size of sentences
        Transform each sentence into a sequence of idx
        '''
        indice = np.random.choice(len(sents), batch_size)
        sents = sents[indice]
        labels = labels[indice]
        #sent_vecs, sent_lens = self.create_sent_idx(sents)
        sent_vecs, sent_lens = self.create_sent_idx(sents)
        return sent_vecs, labels, sent_lens
        #return self.create_sent_idx(sents), labels, sent_lens
    
    
    def create_sent_idx(self, sents):
        '''
        Map sents into idx
        '''
        sents_lens = list(map(self.sent2idx, sents))
        sents_idx, sents_lens = zip(*sents_lens)
        return sents_idx, sents_lens
        
        
    def sent2idx(self, sent):
        '''Map a sentence into a sequence of idx'''
        sent_idx = []
        words = self.sent_split(str(sent))
        lens = len(words)
        ##Cut long sentences
        if lens > self.max_sent_len:
            words = words[:self.max_sent_len]
            lens = self.max_sent_len
        for w in words:
            idx = self.word_to_idx.get(w)
            idx = idx if idx else self.word_to_idx['<unk>']
            sent_idx.append(idx)
        ###Pad short sentences
        for i in np.arange(lens, self.max_sent_len):
            idx = self.word_to_idx['<pad>']
            sent_idx.append(idx)
        return sent_idx, lens
    
    def generate(self, batch_size=64):
        if self.is_training:
            sent_vecs, sent_labels, lengths = self.generate_samples(self.data, 
                                                               self.labels,
                                                              batch_size)
        else:
            start = self.index
            end = start + batch_size
            if end > len(self.data):
                print('Out of sample size')
                self.index = 0
            sents = self.data[start:end]
            sent_labels = self.labels[start:end]
            sent_vecs, lengths = self.create_sent_idx(sents)
            self.index = end
        return sent_vecs, sent_labels, lengths

In [93]:
#from sents_handler import generate_samples
import numpy as np

train_gs = generate_samples(np.array(text_train), np.array(y_train), word_to_idx, 20, False)
#sent_vecs, sent_labels, lengths = train_gs.generate(32)

In [94]:
with tf.Session() as sess:
    saver.restore(sess, conf.ckpt_file)
    #Get the contextualized representation
    #train_gs = generate_samples(np.array(text_train), np.array(y_train), word_to_idx, 20, False)
    sent_vecs, sent_labels, lengths = train_gs.generate(32)
    out_layer = sess.run(model.out_layer, feed_dict={model.X:sent_vecs})
    
    out_layer.shape

INFO:tensorflow:Restoring parameters from ckpt/vocab2000_embed200_filters64_batch32_layers1_block2_fdim5/model.ckpt


In [96]:
max_word_len = out_layer.shape[1]
from sentiment_analysis import CNN_Model_Pretrained_Emb
class trainConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 200
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class testConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 200
    hidden_size = 250
    batch_size = 64
    layer_size = 2
    
class singleConfig:
    max_doc_len = max_word_len
    label_size = 2
    embed_size = 200
    hidden_size = 250#hidden size for hidden state of rnn
    batch_size = 1
    layer_size = 2


In [97]:
import tensorflow as tf
graph_cnn = tf.Graph()
#Create models for training and testing data
with graph_cnn.as_default():
    initializer = tf.random_uniform_initializer(-0.02, 0.02)
    with tf.name_scope('train'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            train_model = CNN_Model_Pretrained_Emb(trainConfig)
            saver=tf.train.Saver()
    with tf.name_scope('test'):
        #Set different models for different buckets
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            test_model = CNN_Model_Pretrained_Emb(testConfig, False)
            single_model = CNN_Model_Pretrained_Emb(singleConfig, False)



Model Initialized!
Model Initialized!
Model Initialized!


In [ ]:
import time, os
epochs = 2
#train_chunk_num = 10
file = "ckpt_cnn_pretrained_emb/cnn.ckpt"
with tf.Session(graph=graph_cnn) as sess:
    #Initialize parameters
    init = tf.global_variables_initializer()
    if not os.path.exists("ckpt_cnn_pretrained_emb"):
        os.mkdir('ckpt_cnn_pretrained_emb')
    if os.path.exists("ckpt_cnn_pretrained_emb/cnn.ckpt.index"):
        saver.restore(sess, file)
    else:
        sess.run(init)
    start_time = time.time()
    for m in range(epochs):
        for i in range(train_chunk_num):
            #sess.run(tf.assign(learning_rate, 0.002*((0.98)**m)))
            x, y, lengths = train_gs.generate(trainConfig.batch_size)
            feed_dict = {train_model.x:x, train_model.y:y, train_model.lengths:lengths}
            l, _ = sess.run([train_model.cost, train_model.optimize], feed_dict=feed_dict)
            if i%100 == 0:
                print('Loss:', round(l, 4))
        end_time = time.time()
        print('Epoch', m, 'time:{:.2f}'.format(end_time - start_time))
        
    saver.save(sess,'ckpt_cnn_pretrained_emb/cnn.ckpt')

In [ ]:
#Calculate Testing Accuracy
with tf.Session(graph=graph_cnn) as sess:
    print('Testing...')
    count = 0
    #saver = tf.train.import_meta_graph('ckpt_cnn/cnn.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('ckpt_cnn_pretrained_emb/'))
    print('Parameters restored')
    start_time = time.time()
    test_gs = generate_samples(np.array(test_processed), np.array(test_labels), False)
    for _ in range(test_chunk_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(testConfig.batch_size)
        feed_dict = {test_model.x:x, test_model.y:y, test_model.lengths:lengths}
        n = sess.run(test_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    for _ in range(remain_num):
        #Traverse each data
        x, y, lengths = test_gs.generate(1)
        feed_dict = {single_model.x:x, single_model.y:y, 
                     single_model.lengths:lengths}
        n = sess.run(single_model.correct_num, feed_dict=feed_dict)
        count += np.sum(n)
    end_time = time.time()
    print('Testing Time:{:.2f}'.format(end_time - start_time))
    print(count*1.0/len(test_processed))